In [ ]:
# リスト 4.2.1 NLU呼び出し用インスタンス生成（説明コメント付き・.envから資格情報読込版）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の Python SDK を用いた
#     クライアント(NaturalLanguageUnderstandingV1)の初期化を行う。
#   ・APIキー/URL はハードコードせず、.env（またはOS環境変数）から安全に取得する。
#
# セキュリティ/運用の要点:
#   ・ソースコード/リポジトリに認証情報をベタ書きしない（.env を .gitignore に追加する）。
#   ・本番/開発で異なる資格情報を使い分ける場合、.env を切り替えるだけで済む構成にする。
#   ・API バージョンは後方互換に注意。必要に応じて環境変数で上書き可能にしておく。
#
# 事前準備:
#   1) python-dotenv を使用する場合はインストール:
#        pip install python-dotenv
#   2) プロジェクト直下に .env を作成（例）
#        WATSON_NLU_APIKEY=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
#        WATSON_NLU_URL=https://api.jp-tok.natural-language-understanding.watson.cloud.ibm.com/instances/xxxxxxxx
#        # （任意）バージョンを固定したい場合
#        # WATSON_NLU_VERSION=2021-08-01
#
#   3) .gitignore に以下を追加して .env をコミットしない:
#        .env
# =============================================================================

import os

# --- .env 読み込み（存在すれば） ------------------------------------------------
try:
    from dotenv import load_dotenv

    # override=False: 既にOS環境に同名キーがある場合は .env を優先しない（誤上書きを防ぐ）
    load_dotenv(override=False)
except ModuleNotFoundError:
    # python-dotenv 未導入でも、OS環境変数から直接取得できるため致命的エラーにはしない
    pass

# --- 環境変数から資格情報を取得 -------------------------------------------------
#   ・キー名の冗長化: 利用環境により命名が違っても拾えるようフォールバックを用意
NLU_APIKEY = (
    os.getenv("WATSON_NLU_APIKEY")
    or os.getenv("NLU_APIKEY")
    or os.getenv("IBM_WATSON_NLU_APIKEY")
)

NLU_URL = (
    os.getenv("WATSON_NLU_URL")
    or os.getenv("NLU_URL")
    or os.getenv("IBM_WATSON_NLU_URL")
)

# 任意: バージョンは環境変数で上書き可能。指定が無ければ従来互換のデフォルトにする
NLU_VERSION = os.getenv("WATSON_NLU_VERSION", "2019-07-12")
# ※ IBM の推奨最新版に合わせたい場合は適宜更新（例: "2021-08-01"）。SDK/サービス側の互換に注意。

# --- バリデーション（早期失敗で原因を明確化） ---------------------------------
missing = []
if not NLU_APIKEY:
    missing.append("WATSON_NLU_APIKEY")
if not NLU_URL:
    missing.append("WATSON_NLU_URL")

if missing:
    raise RuntimeError(
        "NLU の資格情報が不足しています。以下の環境変数を .env または OS 環境に設定してください: "
        + ", ".join(missing)
    )

# --- IBM Watson NLU SDK の初期化 ------------------------------------------------
# SDKのimportは資格情報の検証後に行ってもよい（起動時エラーの責務分離）
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# 旧スタイルのサブモジュール（v1のエクスポート）は必要に応じて:
# from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions, EntitiesOptions

# 認証オブジェクト（IAM）
authenticator = IAMAuthenticator(NLU_APIKEY)

# クライアント生成:
#  - version: API 呼び出しのバージョン日付。サービス仕様と整合する値を用いること。
#  - authenticator: IAMAuthenticator を渡す。
nlu = NaturalLanguageUnderstandingV1(version=NLU_VERSION, authenticator=authenticator)

# サービスURLの設定（インスタンスごとに異なる）
nlu.set_service_url(NLU_URL)

# --- 動作確認の雛形（任意・コメントアウト） -----------------------------------
# from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions
# try:
#     resp = nlu.analyze(
#         text="今日はいい天気ですね。Watsonでキーワード抽出を試します。",
#         features=Features(keywords=KeywordsOptions(limit=3))
#     ).get_result()
#     # print(resp)  # 返却JSON（実運用ではログに個人情報を出さないこと）
# except Exception as e:
#     # ネットワーク/認証/権限/リージョン誤りなどの例外をここで検知
#     raise

In [ ]:
# リスト 4.2.2 NLU呼び出し用共通関数（詳細コメント付き）
# =============================================================================
# 役割:
#   - 事前に初期化済みの IBM Watson NLU クライアント（変数: nlu）を用いて
#     テキスト解析 API を呼び出し、返却 JSON から関心のキーのみを取り出す。
#
# 使い方の要点（理論・API設計の観点）:
#   - IBM Watson NLU では「どの分析を実行するか」を Features オブジェクトで宣言する
#     （例: EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions などを束ねる）。
#   - `nlu.analyze(...)` は HTTP 経由で非同期実行され、`.get_result()` で Python dict(JSON) を返す。
#   - 本関数はその dict から `key` で指定されたトップレベルの要素（例: "entities", "keywords"）を返す。
#     * 存在しないキーを指定すると KeyError となる（後述の安全版で緩和可能）。
#   - ネットワーク障害や認証失敗、レート制限などで例外が発生し得るため、呼び出し側で try/except を推奨。
#
# 例:
#   from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions
#   feats = Features(entities=EntitiesOptions(limit=5, sentiment=False, emotion=False))
#   entities = call_nlu("今日はいい天気ですね。", feats, key="entities")
#   # entities は List[dict]（各エンティティのスパン、タイプ、スコア等を含む）
#
# 注意（実務）:
#   - 入力テキストの長さ・言語・レート制限などサービス側の上限に留意（大きな文書は要分割）。
#   - API のバージョンやリージョンにより挙動が異なる場合があるため、.env で設定を一元管理する。
# =============================================================================

from typing import Any, Dict, List, Union

# Features 型は実行時には SDK から供給されるため、型ヒントとしての参照に留める
# （循環 import を避けるためのフォワード参照も可能）
try:
    from ibm_watson.natural_language_understanding_v1 import (
        Features,
    )  # 型ヒント用（任意）
except Exception:  # ランタイムに存在しない場合でも関数本体の実行には影響しない
    Features = Any  # フォールバック（型チェックツール向けの便宜）


def call_nlu(
    text: str, features: "Features", key: str
) -> Union[Dict[str, Any], List[Any], Any]:
    """
    IBM Watson NLU を呼び出し、返却 JSON から指定キーの要素を取り出して返す薄いヘルパー。

    引数:
        text (str): 解析対象テキスト（UTF-8想定）
        features (Features): 実行する分析機能の宣言（EntitiesOptions/KeywordsOptions 等を束ねたもの）
        key (str): 返却 JSON のトップレベルキー（例: 'entities', 'keywords', 'sentiment', 'categories' など）

    戻り値:
        任意（Union[dict, list, Any]):
            返却 JSON の `key` に対応する値（存在しないキーを指定すると KeyError）

    例外:
        - ネットワーク障害や認証エラー時に SDK 由来の例外が送出される。
        - `key` が返却 JSON に無い場合は KeyError。

    設計メモ:
        - 本関数は「最小限の責務」に留めている（バリデーション/リトライ/ログは呼び出し側で制御）。
        - 実運用では、レート制限や一時的なネットワーク失敗に備えたリトライ・指数バックオフ、
          ならびに返却スキーマ検証（pydantic/dataclasses 等）を併用することを推奨。
    """
    response = nlu.analyze(text=text, features=features).get_result()
    return response[key]


# -----------------------------------------------------------------------------
# （任意）安全版: KeyError の回避・既定キー・軽いバリデーション・簡易リトライを追加する例
# -----------------------------------------------------------------------------
# from ibm_cloud_sdk_core.api_exception import ApiException
# import time
#
# def call_nlu_safe(
#     text: str,
#     features: "Features",
#     key: str | None = None,
#     retries: int = 2,
#     backoff_sec: float = 0.8,
# ) -> Any:
#     """
#     - KeyError を避け、key 未指定時は生の JSON を返す。
#     - 一時的エラーに対して簡易リトライを行う。
#     """
#     last_err = None
#     for attempt in range(retries + 1):
#         try:
#             resp = nlu.analyze(text=text, features=features).get_result()
#             if key is None:
#                 return resp
#             return resp.get(key, None)  # 無ければ None（呼び出し側で判定しやすい）
#         except Exception as e:  # ApiException を個別に握るとより良い
#             last_err = e
#             if attempt < retries:
#                 time.sleep(backoff_sec * (2 ** attempt))  # 指数バックオフ
#                 continue
#             raise last_err
# -----------------------------------------------------------------------------

In [ ]:
# リスト 4.2.3 エンティティ抽出機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「エンティティ抽出」を用いて、
#     テキスト中から固有表現（人物・組織・地名・日付など）を検出し、JSON 形式で確認する。
#
# 背景/理論:
#   ・エンティティ抽出 (Named Entity Recognition; NER) は、文中の名詞句をカテゴリ（PERSON, LOCATION 等）
#     にラベル付けするタスク。以降の関係抽出・イベント抽出・検索インデキシングの基礎特徴となる。
#   ・Watson NLU は言語自動判定を行うが、テキストが短い/混在言語のときは誤判定に注意。
#     必要に応じて features に `language="ja"` を付けるか、分析APIの別引数で明示指定する設計もあり。
#
# 事前準備:
#   ・既に .env から資格情報を読み込み、`nlu` クライアントが初期化済み（リスト 4.2.1）。
#   ・共通関数 `call_nlu(text, features, key)` が定義済み（リスト 4.2.2）。
#
# 出力:
#   ・`entities` キー配下に、抽出されたエンティティの配列（各要素は text, type, relevance など）を返す。
#   ・`ensure_ascii=False` で日本語を可読な形で表示。
# =============================================================================

# （補助）本セルだけで実行する場合に備えた import
#  * 既にインポート済みなら重複定義は無害
import json
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions

# 対象テキスト
# - 人物: 「安倍首相」「トランプ氏」
# - 日付/時間: 「昨日」
# - 施設/場所: 「大阪の国際会議場」
# こうした固有表現が NER の検出対象になる。
text = "安倍首相はトランプ氏と昨日、大阪の国際会議場で会談した。"

# 機能として「エンティティ抽出機能」を利用
# -----------------------------------------------------------------------------
# EntitiesOptions の主なオプション例（必要に応じて拡張）:
#   - limit: 返却件数上限
#   - mentions: エンティティの各出現箇所（スパン）も含めるか
#   - sentiment/emotion: エンティティごとの感情/極性分析を併せて実施
#   - model: カスタム NLU モデルを指定（学習済みがある場合）
# ここでは既定（全件・最小限情報）で実行する。
features = Features(
    entities=EntitiesOptions(
        # limit=10,
        # mentions=True,
        # sentiment=False,
        # emotion=False,
        # model="<your-custom-model-id>"
    )
)

# 共通関数呼び出し
# - 第3引数 "entities" は NLU 返却 JSON のトップレベルキー。
# - 例外（ネットワーク/認証/429 等）は呼び出し側で捕捉する設計（詳細はレビュー参照）。
ret = call_nlu(text, features, "entities")

# 結果の表示
# - 抽出結果は配列（List[dict]）。各要素には "type", "text", "relevance", "count",
#   場合によって "disambiguation"（同名異義の分解）などが含まれる。
# - 日本語を読みやすくするため ensure_ascii=False。
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張ヒント:
#   ・短文で誤検出が起きる場合は、前後文脈を足す/ドメイン固有語を事前正規化する/句点での分割を調整する。
#   ・固有表現のカテゴリ粒度（例: "人物肩書" vs "人名"）はサービス側モデル依存。
#   ・後続処理（例: ES へのインデックス、グラフDB でのノード登録）では、
#     "normalized_text"（正規化表記）や "disambiguation.dbpedia_resource" 等の正規IDをキーに採用すると安定。
# =============================================================================

In [ ]:
# リスト 4.2.4 関係抽出機能の呼び出し（説明コメント付き）
# =============================================================================
# 目的:
#   ・IBM Watson Natural Language Understanding (NLU) の「関係抽出（Relation Extraction）」機能を用いて、
#     文中のエンティティ間の関係（主語-述語-目的語 等の関係ラベル/役割）を検出し、JSON で確認する。
#
# 背景/理論:
#   ・関係抽出は、まず NER（固有表現抽出）で候補エンティティを同定 → そのペア（または n-項）の
#     間に成り立つ意味関係をラベリングするタスクである。
#   ・一般的には (subject, relation, object) の三つ組や、argument 役割（ARG0/ARG1 等）として返る。
#   ・Watson NLU では `relations` を有効化すると、内部でエンティティ・句構造を解析し、
#     「開催地」「所属」「位置」などの関係を推定する（具体的ラベルはモデル依存）。
#
# 実務上の要点:
#   ・短文や文脈の乏しい文では関係の自信度が低くなりやすい。必要なら文脈を付与して精度を上げる。
#   ・日本語判定が揺れると解析器の選択が変わり結果が不安定になるため、必要に応じて language="ja" を
#     明示指定する（本スニペットは共通関数 call_nlu をそのまま使うため引数追加はしていない）。
#   ・返却 JSON はモデル/バージョンで微細に変化し得るため、下流でのキー存在チェックを推奨。
#
# オプション設計のヒント:
#   ・RelationsOptions には（環境により）`model` などの指定が可能（独自学習モデルがある場合に利用）。
#   ・関係抽出は NER の質に依存するため、同時に EntitiesOptions(mentions=True) で可視化すると
#     デバッグしやすい（どのスパンが関係の対象になったか追跡できる）。
#
# 依存:
#   ・前段で NLU クライアント `nlu` が初期化済み（.env から資格情報読込）であること（リスト 4.2.1）。
#   ・共通関数 `call_nlu(text, features, key)` が定義済みであること（リスト 4.2.2）。
# =============================================================================

# 最小限の import（このセル単体での実行にも耐えるようにしておく）
import json
from ibm_watson.natural_language_understanding_v1 import Features, RelationsOptions

# ※ call_nlu は前段で定義済みの想定。未定義なら call_nlu_safe を用意するか、nlu.analyze を直接呼ぶ。

# 対象テキスト
# - エンティティ候補: 「このイベント」（抽象イベント）、「東京」（地名）、「国立競技場」（施設）
# - 期待される関係例: "開催地(location_of_event)" や "場所(at)" に相当する関係ラベル（モデル依存）
text = "このイベントは東京の国立競技場で開催されました。"

# 機能として「関係抽出機能」を利用
# -----------------------------------------------------------------------------
# RelationsOptions の主な指定例（必要に応じて利用）:
#   - model="<custom-model-id>"   : カスタム学習済み関係抽出モデルがある場合に指定
#   - language="ja"               : analyze 引数に渡すのが一般的（call_nlu を拡張するか、直接呼び出す）
# ここでは既定の関係抽出を用いる。
features = Features(
    relations=RelationsOptions(
        # model="<your-custom-relations-model-id>"  # 例: カスタムモデルがある場合
    )
)

# 共通関数呼び出し
# - "relations" キー配下に、検出された関係の配列が返る。
# - 各要素は "type"（関係ラベル）、"arguments"（関係を構成するエンティティ/スパンと役割）、
#   "sentence"（対象文）などを含むことが多い（モデル/バージョンに依存）。
ret = call_nlu(text, features, "relations")

# 結果の表示
# - ensure_ascii=False: 日本語を可読表示。
# - 返却例イメージ（参考/実行環境で変動）:
#   [
#     {
#       "type": "located_at",
#       "sentence": "このイベントは東京の国立競技場で開催されました。",
#       "arguments": [
#         {"text": "このイベント", "entities": [...], "location": {...}, "role": "subject"},
#         {"text": "国立競技場",   "entities": [...], "location": {...}, "role": "object"}
#       ]
#     }
#   ]
print(json.dumps(ret, indent=2, ensure_ascii=False))

# =============================================================================
# 注意/拡張:
#   ・誤検出/過検出が見られる場合:
#       - 文を短く分割しすぎない（係り受け情報が失われる）
#       - 補足文脈を足す（主語省略の補完）
#       - NER の mentions を併用してスパンを確認（関係の引数抽出が適切か検証）
#   ・スキーマの安定運用:
#       - 下流では "type" と "arguments[*].role/text" を必須キーとして扱い、欠落時はスキップ/要再解析。
#       - ログには原文を残さず、必要最小限のメタ情報（関係タイプ、引数の正規表記）を記録。
#   ・評価:
#       - 開発コーパスで関係タイプごとの適合率/再現率/F1 を算出し、文体差（ニュース/ブログ/広報）に対する頑健性を確認。
# =============================================================================